In [1]:
%run base.ipynb

In [2]:
# unit quaternion from euler angles

euler2qf = Function('euler2q', [eul], [euler2q(eul)])
phi_f = 10*(pi/180)
psi_f = 30*(pi/180)
theta_f = -20*(pi/180)
print(euler2qf([phi_f, theta_f, psi_f]))



# euler angles from a unit quaternion

q2eulerf = Function('q2euler', [uq], [q2euler(uq)])
q_f = np.array([0.9437, 0.1277, -0.1449, 0.2685])
print(q2eulerf(q_f))


# c , cpp or matlab code generation for forward dynamics
euler2qf.generate("euler2qf.c")
os.system(f"gcc -fPIC -shared euler2qf.c -o libeuler2qf.so")

q2eulerf.generate("q2eulerf.c")
os.system(f"gcc -fPIC -shared q2eulerf.c -o libq2eulerf.so")

[0.943714, 0.127679, -0.144878, 0.268536]
[0.174579, -0.34912, 0.523523]


0

<h1>Kinematics ned Transformation Matrix</h1>

In [3]:
Kinematics = kin()
Jk = Kinematics.J
J_inv = Kinematics.J_INV
print(J_inv)
J_inv_func = Function('J_inv', [eul], [J_inv]) # for numerical & symbolic use

@1=0, 
[[(cos(psi)*cos(thet)), (sin(psi)*cos(thet)), (-sin(thet)), @1, @1, @1], 
 [(((cos(psi)*sin(thet))*sin(phi))-(sin(psi)*cos(phi))), ((cos(psi)*cos(phi))+((sin(phi)*sin(thet))*sin(psi))), (cos(thet)*sin(phi)), @1, @1, @1], 
 [((sin(psi)*sin(phi))+((cos(psi)*cos(phi))*sin(thet))), (((sin(thet)*sin(psi))*cos(phi))-(cos(psi)*sin(phi))), (cos(thet)*cos(phi)), @1, @1, @1], 
 [@1, @1, @1, 1, 00, (-sin(thet))], 
 [@1, @1, @1, 00, cos(phi), (cos(thet)*sin(phi))], 
 [@1, @1, @1, 00, (-sin(phi)), (cos(thet)*cos(phi))]]


In [4]:
# example usage of J mat
jinv = np.array(J_inv_func([0.13,0.17,0.1]))
jinv

array([[ 0.98066095,  0.09839429, -0.16918235,  0.        ,  0.        ,
         0.        ],
       [-0.07716877,  0.98879774,  0.12776544,  0.        ,  0.        ,
         0.        ],
       [ 0.17985851, -0.11223898,  0.9772683 ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        -0.16918235],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.99156189,
         0.12776544],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.12963414,
         0.9772683 ]])

In [5]:
# reverse scipy convention
def q_reverse(q):
    x, y, z, w = q
    return np.array([w, x, y, z])

def eul_reverse(eul):
    phi, thet, psi = eul
    return np.array([psi, thet, phi])

In [6]:
# state transformations
J_ = Kinematics.J
Jq_ = Kinematics.Jq

J_func = Function('J_', [eul], [J_])
Jq_func = Function('Jq_', [uq], [Jq_])

vr_v = np.array([0, 0.2, 0.0, 0.5, 0, 0.1])
r = R.from_euler("ZYX", (10, 10, 20), degrees=True)
eul_v = r.as_euler("ZYX")
uq_v = r.as_quat()

# print(uq_v)

 #, r.as_matrix()

Jo = J_func(eul_reverse(eul_v))
Jqo = Jq_func(q_reverse(uq_v))

To = Jo[3:,3:]
Tq = Jqo[3:,3:]

# state discrete time propagation in euler
h = 0.05
wk_next = eul_reverse(eul_v) + h*To@vr_v[3:]

# state discrete time propagation and normalization in unit quaternion
qk_next = q_reverse(uq_v)+ h*Tq@vr_v[3:]
norm_qk_next = qk_next/sqrt(qk_next.T@qk_next)


# response
print(wk_next)
print(norm_qk_next)


#checking for property 8.17 --> eye(3)
4*(Tq.T@Tq) 

[0.374894, 0.172823, 0.179304]
[0.97633, 0.177319, 0.101042, 0.0716117]


DM(
[[1, 3.91182e-19, 2.28221e-18], 
 [3.91182e-19, 1, 1.2201e-17], 
 [2.28221e-18, 1.2201e-17, 1]])

In [7]:
# compare euler and quaternion results
rr = R.from_euler("ZYX", (0.179304, 0.172823, 0.374894), degrees=False)
q_reverse(rr.as_quat()) , norm_qk_next

# as expected both quaternion and euler produce same response

(array([0.97632635, 0.1773187 , 0.10107384, 0.07161717]),
 DM([0.97633, 0.177319, 0.101042, 0.0716117]))